<a href="https://colab.research.google.com/github/jasonhouse1/TKU-AI-411631327/blob/main/0327.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

#things we need for tensorflow

import numpy as np
import tflearn
import tensorflow as tf
import random


In [ ]:
#import chatbot data file
import json

with open('/content/sample_data/intent') as json_data:
  intents = json.load(json_data)

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']

#loop through each sentence in our intents patterns

for intent in intents['intents']:
  for pattern in intent ['patterns']:
    #tokenize each word in the sentence
    w = nltk.word_tokenize(pattern)
    #add to word list
    words.extend(w)

    #add to documnets in our corpus
    documents.append(w)

    #add to classes list
    if intent['tag'] not in classes :
      classes.append(intent['tag'])

# stem ans lower each woord and remove duplicaes
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicates
classes = sorted (list(set(classes)))

print(len(documents),"sentences ")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(classes)
print(X.shape)
print(type(X))
print(X.toarray())
# training = np.array(X)
# print(training)
#create train and test lists
train_x = list(X[:,0])
print("train shape",len(train_x))
test = list(X[:,1])
print("test shape",len(test))

In [3]:
(train_x[8])

NameError: name 'train_x' is not defined

In [ ]:
train_x.shape[0]

In [ ]:
#reset underlying graph data
tf.reset_default_graph()
#Build neural network
net = tflearn.input_data(shape=[None,1])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,1,activation='softmax')
net = tflearn.regression(net)
#Define model and setup tensorboard

model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')
#start traiing (apply gradient descent algorithm)
model.fit(train_x,test,n_epoch=1000,batch_size=8,show_metric=True)
#model.save('model.tflearn')

In [ ]:
#create our training data
training = []
output = []

#create an empty array for our output
output_empty = [0]* len(classes)

#traning set , bag of words for each sentence

for doc in documents :
  # initialize our bag of words
  bag =[]
  #list of tokenized words for the pattern

  pattern_words = doc[0]

  #stem each word
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  #create our bag of words array
  for w in words:
    bag.append(1)

   #output is a '0' for each tag and '1' for current tag
  output_row = list(output_empty)
  output_row[classes.index(doc[0])] = 1
  training.append([bag,output_row])

#shuffle our feature and turn into np.array

random.shuffle(training)
training = np.array(training)

#create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])



